# Test 1: Clay Baseline

Interactive exploration of the monolithic mixed FEM for clay soil.
Reproduces Gatti et al. 2024 Figure 6 with adjustable parameters.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from nitsche_signorini import CLAY
from nitsche_signorini.runners import run_monolithic

In [ ]:
# Adjustable parameters
soil = CLAY
H = 5.0          # Column height [m]
maxh = 0.05      # Mesh size [m]
dt = 5 * 3600    # Time step [s]
t_final = 50 * 3600  # Final time [s]
p_over_Ks = 10   # Rain intensity ratio
p_rain = p_over_Ks * soil.K_s

snap_times = np.arange(dt, t_final + 1, dt)

In [ ]:
z, snaps, info = run_monolithic(
    soil, H, maxh, dt, t_final, p_rain, snap_times)

nits = info['newton_its']
print(f"Steps: {info['n_steps']}, Newton: mean={nits.mean():.1f}, max={nits.max()}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 7))
cmap = plt.cm.viridis
n_prof = len(info['psi_snapshots'])

for i, (t_s, psi_s) in enumerate(info['psi_snapshots']):
    c = cmap(i / max(n_prof - 1, 1))
    axes[0].plot(psi_s, z, color=c, lw=1.2, label=f"{t_s/3600:.0f}h")

for i, (t_s, qz_s) in enumerate(info['qz_snapshots']):
    c = cmap(i / max(n_prof - 1, 1))
    axes[1].plot(qz_s, z, color=c, lw=1.2, label=f"{t_s/3600:.0f}h")

axes[0].axvline(0, color='grey', ls=':', lw=0.5)
axes[0].set(xlabel='psi [m]', ylabel='z [m]', title='Pressure head')
axes[0].legend(fontsize=6, ncol=2)
axes[1].set(xlabel='q_z [m/s]', ylabel='z [m]', title='Vertical Darcy flux')
axes[1].legend(fontsize=6, ncol=2)

fig.suptitle(f'Clay, p/K_s = {p_over_Ks}, dt = {dt/3600:.0f}h')
fig.tight_layout()
plt.show()